# SVM 8 emotions

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

Load data and scale it
Remove highly correlated features
Create train and test splits

In [2]:
# data loading
dataset = pd.read_csv('../dataset/labeled_8_combined_dataset.csv', index_col=0)

data = dataset.iloc[:, :-4]
labels = dataset.iloc[:, -1]
unique_labels = np.unique(labels)

# scaling the data
data_scaled = StandardScaler().fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)

# removing correlated features
corr = data.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
data.drop(to_drop, axis=1, inplace=True)

# creating train and test splits
data_train, data_test, labels_train, labels_test = train_test_split(data_scaled, labels, test_size=0.1, random_state=0)

# ECG

Extract only ecg related features

In [3]:
ecg_columns = [col for col in data.columns if 'ECG' in col or 'HRV' in col]

ecg_data_train = data_train[ecg_columns]
ecg_data_test = data_test[ecg_columns]

Define the model and train it

In [4]:
ecg_svc = SVC(
    C=1.0,
    kernel='rbf',
    gamma=0.001,
    max_iter=-1,
    decision_function_shape='ovr'
)

In [5]:
ecg_svc.fit(ecg_data_train, labels_train)
ecg_predictions = ecg_svc.predict(ecg_data_test)

In [6]:
ecg_cr = classification_report(labels_test, ecg_predictions, target_names=unique_labels, zero_division=0, digits=4, output_dict=True)
print(classification_report(labels_test, ecg_predictions, target_names=unique_labels, zero_division=0, digits=4))

              precision    recall  f1-score   support

  Ekscytacja     0.0000    0.0000    0.0000        85
   Neutralna     0.1878    0.9949    0.3160       198
  Odprezenie     0.0000    0.0000    0.0000       117
      Radosc     0.3750    0.0196    0.0373       153
      Smutek     0.0000    0.0000    0.0000        77
      Wstret     0.0000    0.0000    0.0000       182
  Zdziwienie     0.0000    0.0000    0.0000       102
       Zlosc     0.0000    0.0000    0.0000       143

    accuracy                         0.1892      1057
   macro avg     0.0703    0.1268    0.0442      1057
weighted avg     0.0895    0.1892    0.0646      1057



# EDA

Extract only ecg related features

In [7]:
eda_columns = [col for col in data.columns if 'EDA' in col or 'SCR' in col]

eda_data_train = data_train[eda_columns]
eda_data_test = data_test[eda_columns]

Define the model and train it

In [8]:
eda_svc = SVC(
    C=1.0,
    kernel='rbf',
    gamma=0.001,
    max_iter=-1,
    decision_function_shape='ovr'
)

In [9]:
eda_svc.fit(eda_data_train, labels_train)
eda_predictions = eda_svc.predict(eda_data_test)

In [10]:
eda_cr = classification_report(labels_test, eda_predictions, target_names=unique_labels, zero_division=0, digits=4, output_dict=True)
print(classification_report(labels_test, eda_predictions, target_names=unique_labels, zero_division=0, digits=4))

              precision    recall  f1-score   support

  Ekscytacja     0.0000    0.0000    0.0000        85
   Neutralna     0.1890    0.9899    0.3174       198
  Odprezenie     0.0000    0.0000    0.0000       117
      Radosc     0.0000    0.0000    0.0000       153
      Smutek     0.0000    0.0000    0.0000        77
      Wstret     0.1500    0.0165    0.0297       182
  Zdziwienie     0.0000    0.0000    0.0000       102
       Zlosc     0.0000    0.0000    0.0000       143

    accuracy                         0.1883      1057
   macro avg     0.0424    0.1258    0.0434      1057
weighted avg     0.0612    0.1883    0.0646      1057



# Combined

In [11]:
svc = SVC(
    C=1.0,
    kernel='rbf',
    gamma=0.001,
    max_iter=-1,
    decision_function_shape='ovr'
)

In [12]:
svc.fit(data_train, labels_train)
predictions = svc.predict(data_test)

In [13]:
cr = classification_report(labels_test, predictions, target_names=unique_labels, zero_division=0, digits=4, output_dict=True)
print(classification_report(labels_test, predictions, target_names=unique_labels, zero_division=0, digits=4))

              precision    recall  f1-score   support

  Ekscytacja     0.0000    0.0000    0.0000        85
   Neutralna     0.1889    0.9596    0.3156       198
  Odprezenie     0.0000    0.0000    0.0000       117
      Radosc     0.3333    0.0196    0.0370       153
      Smutek     0.0000    0.0000    0.0000        77
      Wstret     0.2143    0.0495    0.0804       182
  Zdziwienie     0.0000    0.0000    0.0000       102
       Zlosc     0.0000    0.0000    0.0000       143

    accuracy                         0.1911      1057
   macro avg     0.0921    0.1286    0.0541      1057
weighted avg     0.1205    0.1911    0.0783      1057



# Summary

In [14]:
print(f'''
\tAccuracy:
ECG:\t{ecg_cr['accuracy']}
EDA:\t{eda_cr['accuracy']}
Both:\t{cr['accuracy']}
\tAverage F1:
ECG:\t{ecg_cr['macro avg']['f1-score']}
EDA:\t{eda_cr['macro avg']['f1-score']}
Both:\t{cr['macro avg']['f1-score']}
\tWeighted F1:
ECG:\t{ecg_cr['weighted avg']['f1-score']}
EDA:\t{eda_cr['weighted avg']['f1-score']}
Both:\t{cr['weighted avg']['f1-score']}
''')


	Accuracy:
ECG:	0.1892147587511826
EDA:	0.18826868495742669
Both:	0.19110690633869443
	Average F1:
ECG:	0.04415317258314364
EDA:	0.0433889846474526
Both:	0.0541260997292974
	Weighted F1:
ECG:	0.06458051725464442
EDA:	0.06457228397049428
Both:	0.07831916841894619

